In [23]:
from langchain.tools import tool, ToolRuntime
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent, AgentState
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import wrap_model_call, ModelResponse, ModelRequest
from dataclasses import dataclass

load_dotenv()


OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")


llm = ChatOpenAI(
    model="google/gemini-3-flash-preview",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)

In [24]:
from tavily import TavilyClient
from typing import Dict, Any, Callable
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:
    """Search the web for information"""
    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:
    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [25]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [26]:
@wrap_model_call
def dynamic_tool_call(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Dynamically call tools based on the runtime context"""
    user_role = request.runtime.context.user_role

    if user_role == "internal":
        pass
    else:
        tools = [web_search]
        request = request.override(tools=tools)

    return handler(request)

In [27]:
agent = create_agent(
    model=llm,
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [28]:
response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "internal"}
)

print(response["messages"][-1].content)

There are 275 artists in the database.
